In [ ]:
# CÉLULA 1: Imports e Configurações Iniciais
import os
import hashlib
import logging
from pathlib import Path
from datetime import datetime

from minio import Minio
from minio.error import S3Error
import psycopg2
from psycopg2 import sql
from tqdm.notebook import tqdm  # Usar tqdm.notebook para Jupyter

# --- Configurações ---
# Caminho da montagem do Google Drive dentro do contêiner Jupyter
GDRIVE_MOUNT_PATH = Path("/home/jovyan/work/gdrive_local_mount/")

# Configurações do MinIO
MINIO_ENDPOINT = "minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "senhasegura"
MINIO_BUCKET_RAW = "recepcao-raw"

# Configurações do PostgreSQL
POSTGRES_HOST = "postgres_db"
POSTGRES_PORT = "5432"
POSTGRES_DB = "postgres"
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "senhasegura"

# Configuração do Logging
LOG_DIR = Path("/home/jovyan/work/logs/")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = LOG_DIR / f"processamento_drive_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)
logger.info("Configurações iniciais e imports carregados.")
logger.info(f"Montagem do Google Drive esperada em: {GDRIVE_MOUNT_PATH}")
logger.info(f"Log será salvo em: {LOG_FILE}")

# Criar tabela de auditoria genérica de downloads (não depende de projetos)
PG_CONN = psycopg2.connect(
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD
)
cursor = PG_CONN.cursor()
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS reception_audit (
        id SERIAL PRIMARY KEY,
        diretorio_origem TEXT NOT NULL,
        caminho_relativo TEXT NOT NULL,
        nome_arquivo TEXT NOT NULL,
        hash_sha256 VARCHAR(64) NOT NULL,
        tamanho_bytes BIGINT NOT NULL,
        data_processamento TIMESTAMPTZ NOT NULL,
        caminho_minio TEXT NOT NULL
    );
    """
)
PG_CONN.commit()
logger.info("Tabela 'reception_audit' verificada/criada com sucesso.")

# CÉLULA 2: Widget de Seleção da Pasta
import ipywidgets as widgets
from IPython.display import display, clear_output

# Inicializa cliente MinIO
client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)
if not client.bucket_exists(MINIO_BUCKET_RAW):
    client.make_bucket(MINIO_BUCKET_RAW)

# Lista todas as pastas no mount do Drive
dir_list = []
for root, dirs, _ in os.walk(GDRIVE_MOUNT_PATH):
    for d in dirs:
        rel = os.path.relpath(os.path.join(root, d), GDRIVE_MOUNT_PATH)
        dir_list.append(rel)

dropdown = widgets.Dropdown(options=sorted(dir_list), description="Pasta:")
button = widgets.Button(description="INICIAR DOWNLOAD")
output = widgets.Output()

def selecionar_pasta(btn):
    with output:
        clear_output()
        print(f"Pasta selecionada: {dropdown.value}")
button.on_click(selecionar_pasta)
display(dropdown, button, output)

# CÉLULA 3: Upload ao MinIO + Persistência no reception_audit
from datetime import datetime

def upload_e_auditar(btn):
    with output:
        clear_output()
        selected = dropdown.value
        print(f"Iniciando upload da pasta '{selected}' para bucket '{MINIO_BUCKET_RAW}'...")

        tarefas = []
        base_dir = GDRIVE_MOUNT_PATH / selected
        for root, _, files in os.walk(base_dir):
            for fname in files:
                path_local = os.path.join(root, fname)
                rel_drive = os.path.relpath(path_local, GDRIVE_MOUNT_PATH)
                obj_path = os.path.join(selected, os.path.relpath(path_local, base_dir))
                tarefas.append((path_local, rel_drive, obj_path))

        for local_path, rel_drive, obj_rel in tqdm(tarefas, desc="Upload & Auditoria"):
            object_name = obj_rel.replace(os.sep, "/")
            client.fput_object(MINIO_BUCKET_RAW, object_name, local_path)

            tamanho = os.path.getsize(local_path)
            sha256 = hashlib.sha256()
            with open(local_path, "rb") as f:
                for chunk in iter(lambda: f.read(8192), b""):
                    sha256.update(chunk)
            hash_hex = sha256.hexdigest()

            # Insere no reception_audit
            cursor.execute(
                """
                INSERT INTO reception_audit (
                    diretorio_origem,
                    caminho_relativo,
                    nome_arquivo,
                    hash_sha256,
                    tamanho_bytes,
                    data_processamento,
                    caminho_minio
                ) VALUES (%s, %s, %s, %s, %s, %s, %s);
                """,
                (
                    dropdown.value,
                    rel_drive,
                    os.path.basename(local_path),
                    hash_hex,
                    tamanho,
                    datetime.utcnow(),
                    object_name
                )
            )
            PG_CONN.commit()

        print(f"Processo concluído: {len(tarefas)} arquivos enviados e auditados.")

button.on_click(upload_e_auditar)


2025-05-29 17:04:58,769 - INFO - Configurações iniciais e imports carregados.
2025-05-29 17:04:58,769 - INFO - Montagem do Google Drive esperada em: /home/jovyan/work/gdrive_local_mount
2025-05-29 17:04:58,769 - INFO - Log será salvo em: /home/jovyan/work/logs/processamento_drive_20250529_170458.log
2025-05-29 17:04:58,775 - INFO - Tabela 'reception_audit' verificada/criada com sucesso.


Dropdown(description='Pasta:', options=('0_PROJETOS WRMELO', '0_PROJETOS WRMELO/0. PROJETO MBA', '0_PROJETOS W…

Button(description='INICIAR DOWNLOAD', style=ButtonStyle())

Output()